# Import libraries

In [2]:
import os
import sys
import tweepy
import requests
from dotenv import load_dotenv
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sys.path.append('../../')
import config

load_dotenv(os.path.join(config.PROJ_PATH, '.env')) 

True

# Set twitter access

In [2]:
def set_twitter_access():
  consumer_key = os.environ["API_KEY"]
  consumer_secret = os.environ["API_KEY_SECRET"]
  access_token = os.environ["ACCESS_TOKEN"]
  access_token_secret = os.environ["ACCESS_TOKEN_SECRET"]

  auth = tweepy.OAuth1UserHandler(
    consumer_key, 
    consumer_secret, 
    access_token, 
    access_token_secret
  )

  api = tweepy.API(auth)

  return api

api = set_twitter_access()


# Create functions to read and save texts and images

In [3]:
def save_tweet_text(tweet:object, text_type='') -> None:
    tweet_timestamp = str(tweet.created_at.strftime("%Y-%m-%d %H-%M-%S"))
    tweet_id = str(tweet.id)
    txt_file_name = os.path.join(config.DATA_PATH_RAW_TEXTS, '{}_{}.txt'.format(tweet_timestamp, tweet_id))
    
    with open(txt_file_name, 'w', encoding="utf-8") as txt_file:
        try:
            if text_type == 'retweet':
                txt_file.write(tweet.retweeted_status.full_text)
            else:
                txt_file.write(tweet.full_text)
            # txt_file.write(tweet.retweeted_status.full_text)
        except AttributeError:
            print('Unable to read text from tweet {}'.format(tweet.id))
            print("=====")

    return None

def DownloadFile(url:str, path_to_save:str) -> None:
    response = requests.get(url)

    if response.status_code == 200:
        with open(path_to_save, 'wb') as f:
                f.write(response.content)
    return None


def save_tweet_image(tweet:object) -> None:
    tweet_timestamp = str(tweet.created_at.strftime("%Y-%m-%d %H-%M-%S"))
    tweet_id = str(tweet.id)

    try:
        for media in tweet.entities.get("media",[{}]):
            #checks if there is any media-entity
            if media.get("type",None) == "photo":
                tweet_media_id = str(media['id'])
                filename = os.path.join(config.DATA_PATH_RAW_IMAGES, '{}_{}_{}.png'.format(tweet_timestamp, tweet_id, tweet_media_id))
                DownloadFile(media["media_url"], filename)
    except AttributeError:
        print('Unable to read medias from tweet {}'.format(tweet_id))
        print("=====")

# Pull tweets

In [4]:
phrases_related_to_vaw = ['que ficar em casa', 'mulher obediente', 'é divorciada', 'Ninguém vai acreditar em você', 
    'Não presta nem pra cozinha', 'Mal sabe lavar uma roupa direito', 'burra', 'interesseira']

In [5]:
def extract_tweets(to_extract, place:str=''):
    query = ""
    if place:
        query += f'place:{place} AND '

    if type(to_extract) is list:
        keywords_query = ' OR '.join(to_extract)
        query += f'({keywords_query})'
    else:
        query += to_extract
    
    print(query)
    tweets_pages = []
    for status in tweepy.Cursor(api.search_tweets,
                                query, 
                                tweet_mode='extended', 
                                lang='pt', 
                                count=3).pages(3):
        tweets_pages.append(status)

    # Read tweets
    for page in tweets_pages:
        for tweet in page:
            save_tweet_text(tweet)
            print(tweet.full_text)

            save_tweet_image(tweet)

            if tweet.coordinates is not None:
                print(tweet.coordinates)
                print(tweet.geo)
                print(tweet.contributors)
                break

extract_tweets(['Interesseira', 'vaca'])

(Interesseira OR vaca)


Forbidden: 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product

In [ ]:
tweets_pages = []
for status in tweepy.Cursor(api.search_tweets,
                            'place:"new york city" OR place:seattle OR place:fd70c22040963ac7', 
                            tweet_mode='extended',
                            count=3).pages(3):
    tweets_pages.append(status)

for page in tweets_pages:
    for tweet in page:

        print(tweet.full_text)

@EverMidnights Don’t give him the attention he wants. They love any kind of attention, including negative
Two of my all time favorite movies released on the same day 41 years ago today. Blade Runner &amp; The Thing are both masterpieces of their genres. https://t.co/0YvTcZ83ud
@CameronJNixon true, can’t see occlusion from this vid
@SlangsOnSports You make baseball so much more fun!
normally i throw out “possible” when debris is visibly being lofted but to each their own https://t.co/AndCKsYVwq
Thank you so much @ondecksoftball!! I’m so grateful I had the opportunity to showcase myself at the Colorado Jamboree today. It was a great way to start my trip before IDT on Wednesday🥎🥎 https://t.co/rEjNvFKM0i
@TranscendCEO @jonnym1ller @RobertKennedyJr @AubreyMarcus That could tip the scales! ⚖️ 🏋️‍♀️
I fucking love punk rock https://t.co/BEOJ7vn8XX
I need to just trust my gut more lol wow


In [ ]:
# Usefull tweet attributes [author(id, name, screen_name, location), user(id, name, screen_name, location), 
#                           geo, place, coordinates]
print(vars(tweet).keys())
print(vars(tweet.user).keys())
print(vars(tweet.place).keys())

dict_keys(['_api', '_json', 'created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'metadata', 'source', 'source_url', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'author', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])
dict_keys(['_api', '_json', 'id', 'id_str', 'name', 'screen_name', 'location', 'description', 'url', 'entities', 'protected', 'followers_count', 'friends_count', 'listed_count', 'created_at', 'favourites_count', 'utc_offset', 'time_zone', 'geo_enabled', 'verified', 'statuses_count', 'lang', 'contributors_enabled', 'is_translator', 'is_translation_enabled', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_background_tile', 'profile_image_url', 'profile_image_url_https', 'profile_banner_url', 'profil

In [ ]:
print(tweet.id) # Id dado tweet (string)
print(tweet.full_text) # Texto (string)
['Interesseira', 'vaca'] # Palavras chaves para extração (list)
print(tweet.created_at) # Data do tweet (timestamp)
print(tweet.user.id_str) # Id do usuário (string)
print(tweet.user.name) # Nome completo (string)
print(tweet.user.screen_name) # Screen_name
print(tweet.user.profile_image_url) # Profile_image_url
print(tweet.user.created_at)# Created_at (timestamp)
print(tweet.place.id) # Id da localização (string)
print(tweet.place.place_type) # Tipo do lugar (string)
print(tweet.place.name) # Nome (string)
print(tweet.place.country) # País (string)
print(tweet.place.country_code) # Código do país (string)
print(tweet.place.bounding_box.coordinates) # Coordenadas (list)


1673095053064163328
I need to just trust my gut more lol wow
2023-06-25 22:25:19+00:00
299153276
dc
DelaneyCoghlan
http://pbs.twimg.com/profile_images/1654174728645660672/6uTq6iV8_normal.jpg
2011-05-15 16:04:41+00:00
fd70c22040963ac7
city
Boulder
United States
US
[[[-105.3017759, 39.953552], [-105.183597, 39.953552], [-105.183597, 40.094411], [-105.3017759, 40.094411]]]


In [ ]:
file_path = os.path.join(config.DATA_PATH_WRANGLE_TWEETS, 'extracted_tweets.parquet')
last_extracted_tweets = pd.read_parquet(file_path)
# if len(last_extracted_tweets) > 0:
#     tweet = last_extracted_tweets.iloc[0].to_dict()
#     last_extracted_tweets = last_extracted_tweets.drop(0).reset_index(drop=True)
#     last_extracted_tweets.to_parquet(file_path)


last_extracted_tweets

,id,text,lang,keywords_extraction,created_at,user_id,user_name,user_screen_name,user_profile_image_url,user_created_at,place_id,place_type,place_name,country,country_code,coordinates
0,1672728421217058816,RT @jjwanstar: Foi essa vagabunda aqui que com...,pt,"b'[""Vagabunda""]'",2023-06-24 22:08:28+00:00,1246271282141896704,lu⁷ 💜 Take Two 🎶 Haegeum 🥢D-DAY 💙,lu_winterflower,http://pbs.twimg.com/profile_images/1670982460...,2020-04-04 03:00:51+00:00,None,None,None,None,None,None
1,1672993375031554048,RT @pedroperegrinaa: lista de CRIMES de que po...,pt,"b'[""Burra""]'",2023-06-25 15:41:17+00:00,748686426947063808,Gabriel Berthier,GabriBerthier,http://pbs.twimg.com/profile_images/1634319430...,2016-07-01 01:15:28+00:00,None,None,None,None,None,None
2,1672993316177080327,@soaareesx @hentaidecarro nossa além de tudo é...,pt,"b'[""Burra""]'",2023-06-25 15:41:03+00:00,1324447067889192964,tailandia do tt,nojodesexo,http://pbs.twimg.com/profile_images/1672105006...,2020-11-05 20:22:53+00:00,None,None,None,None,None,None
3,1672993280710017025,@RomuloBDias Os meios de comunicação tem que e...,pt,"b'[""Burra""]'",2023-06-25 15:40:55+00:00,805645123,Pati Vasconcelos,patimvo,http://pbs.twimg.com/profile_images/1242257318...,2012-09-05 23:44:04+00:00,None,None,None,None,None,None
4,1672993271470071809,"RT @niltoncesarr_: Campanha, ganhe o selo de p...",pt,"b'[""Burra""]'",2023-06-25 15:40:53+00:00,514233754,Bruna (Taylor’s Version),MugiwaraWizard,http://pbs.twimg.com/profile_images/1672781125...,2012-03-04 11:23:40+00:00,None,None,None,None,None,None
5,1672993227408830464,tomara ki eu morra dormindo sou mt burra,pt,"b'[""Burra""]'",2023-06-25 15:40:42+00:00,1663993648155901954,calva,fefecarequinha,http://pbs.twimg.com/profile_images/1671007407...,2023-05-31 19:39:47+00:00,None,None,None,None,None,None
6,1673112847218270208,EU SÓ QUERIA VER MINHA CARTEIRA DE TRABALHO GE...,pt,"b'[""Burra""]'",2023-06-25 23:36:02+00:00,843608327376506881,Brubs,Bubunieli,http://pbs.twimg.com/profile_images/1672003646...,2017-03-19 23:41:13+00:00,None,None,None,None,None,None
7,1673112838376681472,essa vagabunda fica se fazendo se nao é burra ...,pt,"b'[""Burra""]'",2023-06-25 23:36:00+00:00,1360296153053618183,karina,hwaan_08,http://pbs.twimg.com/profile_images/1657768122...,2021-02-12 18:34:22+00:00,None,None,None,None,None,None
8,1673112807296868355,zero paciência pra gente burra,pt,"b'[""Burra""]'",2023-06-25 23:35:52+00:00,1403840307205136385,nano,mnryding,http://pbs.twimg.com/profile_images/1672091450...,2021-06-12 22:23:16+00:00,None,None,None,None,None,None
9,1673112787298447360,Eu e a Carol somos tão burra que a gente tem q...,pt,"b'[""Burra""]'",2023-06-25 23:35:48+00:00,1076592314376732672,Érika Letícia ,erik4_arj,http://pbs.twimg.com/profile_images/1664437929...,2018-12-22 21:36:24+00:00,None,None,None,None,None,None


In [ ]:
pd.read_parquet(config.DATASET_TWEETS_TRAIN_FILE)

,id,text,lang,keywords_extraction,created_at,user_id,user_name,user_screen_name,user_profile_image_url,user_created_at,place_id,place_type,place_name,country,country_code,coordinates,label
0,1673140137805463553,@Leh_bonim @euha_jhenni mano vc é burra ou se faz,pt,"b'[""Burra"",""Pregui\\u00e7osa"",""Manipuladora""]'",2023-06-26 01:24:28+00:00,1296853363603243014,camilly,ttdacamilotaa,http://pbs.twimg.com/profile_images/1665732745...,2020-08-21 16:55:37+00:00,None,None,None,None,None,None,1
1,1673140110055940102,pfv me avisem quando vcs forem mudar de icon p...,pt,"b'[""Burra"",""Pregui\\u00e7osa"",""Manipuladora""]'",2023-06-26 01:24:22+00:00,1660309603869945865,missa pereira da silva,spreenviado,http://pbs.twimg.com/profile_images/1670564997...,2023-05-21 15:41:01+00:00,None,None,None,None,None,None,0
2,1673140237638291458,@viquetooria eu sou meio burra sim victoria,pt,"b'[""Burra"",""Pregui\\u00e7osa"",""Manipuladora""]'",2023-06-26 01:24:52+00:00,2981996553,Catharina,cath_costa,http://pbs.twimg.com/profile_images/1665738676...,2015-01-17 02:03:57+00:00,None,None,None,None,None,None,0
